<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/roof_type_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/roof_type_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**

In [ ]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

## **Import ImageSet and Importer Methods of BRAILS++**

In [ ]:
from brails.types.image_set import ImageSet
from brails import Importer

## **Load Aerial Imagery**
The `RoofShapeClassifier` class is designed to identify roof types of buildings based on aerial imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.

1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, aerial imagery of buildings in Pinellas County, FL, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Pinellas County, FL`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `USA_FootprintScraper`.</strong> The only information required to initialize `USA_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `USA_FootprintScraper` is an `AssetInventory` saved in `usa_fp_data`.

3. <strong>Get a randomly selected `150` building subset of the footprint inventory obtained using `USA_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `atlantic_fp_subset`. For this example, the random seed is set to `75` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleSatellite` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `usa_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/satellite/`.

In [ ]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName", "data": "Pinellas County, FL"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

usa_fp_class = importer.get_class("USA_FootprintScraper")
usa_fp_data = {"length": "ft"}
footprint_scraper = usa_fp_class(usa_fp_data)
usa_fp_inventory = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 150 buildings using a random seed value of 75:
usa_fp_subset = usa_fp_inventory.get_random_sample(150, 75)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_aerial_class = importer.get_class("GoogleSatellite")
google_aerial = google_aerial_class()
images_aerial = google_aerial.get_images(
    usa_fp_subset, "tmp/satellite/")

## **Predict Roof Types of Buildings Using RoofShapeClassifier Module**
`RoofShapeClassifier` includes a pre-trained EfficientNet_V2_M model that has been trained on a custom dataset of 100,000 images. This model is capable of classifying roofs seen in aerial imagery into three categories: `Flat`, `Hip`, and `Gable`.

In [ ]:
importer = Importer()
roof_shape_classifier_class = importer.get_class('RoofShapeClassifier')
roof_shape_classifier = roof_shape_classifier_class()
predictions = roof_shape_classifier.predict(images_aerial)

## **Show Prediction Results**

In [ ]:
print(predictions)